# Notebook: pg_server_extensions
This notebooks shows the installed extensions in the PG server. 
Then it uses the OpenInsight engine to generate a few insights about missing extensions.

## Step 1: Get the list of installed extensions

In [1]:
import sqlalchemy
import pandas as pd
import configparser
import matplotlib.pyplot as plt 
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

# Read from the Config file
config = configparser.ConfigParser() 
config.read_file(open(r'../ipynb.cfg'))

con_str = config.get('con_str', 'PG_AIRBASES') 

engine = sqlalchemy.create_engine(con_str)
sql_command = """
SELECT e.extname AS "Name", e.extversion AS "Version", n.nspname AS "Schema", c.description AS "Description" 
FROM pg_catalog.pg_extension e 
LEFT JOIN pg_catalog.pg_namespace n ON n.oid = e.extnamespace 
LEFT JOIN pg_catalog.pg_description c ON c.objoid = e.oid AND c.classoid = 'pg_catalog.pg_extension'::pg_catalog.regclass 
ORDER BY 1;;
"""

# print("Connecting with engine " + str(engine))
try:
    connection = engine.connect()
    df_pg_server_extensions = pd.read_sql_query(sql_command, connection)
except (Exception, sqlalchemy.exc.SQLAlchemyError) as error:
    print("Error while connecting to PostgreSQL database:", error)

df_pg_server_extensions

,Name,Version,Schema,Description
0,log_fdw,1.2,public,foreign-data wrapper for Postgres log file access
1,pg_cron,1.4,public,Job scheduler for PostgreSQL
2,pg_stat_statements,1.8,public,track planning and execution statistics of all...
3,plpgsql,1.0,pg_catalog,PL/pgSQL procedural language


# Generate the OpenInsights
The list of installed extensions probably doesn't say much to the avg developer.  Therefore the notebook geneates a few insights. 
For ex. If the extension pg_stat_statment doesn't exist, this is bad. Why? we can't analyze the performance of the DB server without it.  

In [2]:
import sys
import pandas as pd
sys.path.append('/Users/itaybraun/Documents/GitHub/useful-postgres-queries/classes')
from OpenInsight import OpenInsight
from OpenInsight import Severity

# Check if 'pg_stat_statements' installed
if 'pg_stat_statements' in df_pg_server_extensions['Name'].values:
    print("pg_stat_statements exists in the 'Name' column.")

    # Create a new OpenInsight object
    open_insight = OpenInsight(
        name="pg_stat_statements extension installed",
        ID="123456",
        category="Extensions",
        status="Healthy",
        # severity=Severity.MEDIUM,
        context="The extension pg_stat_statement installed",
        impact="Performance - without the extesion it is much harder to detect and analyze DB performance"
        # remediation_plan="Configure and enable pg_stat_statements"
    )

    # Access the attributes of the OpenInsight object
    print(f"Name: {open_insight.name}")
    print(f"ID: {open_insight.ID}")
    print(f"Category: {open_insight.category}")
    print(f"Status: {open_insight.status}")
    print(f"Severity: {open_insight.severity.value}")
    print(f"Context: {open_insight.context}")
    print(f"Impact: {open_insight.impact}")
    print(f"Remediation Plan: {open_insight.remediation_plan}")

else:
    print("pg_stat_statements does not exist in the 'Name' column.")


pg_stat_statements exists in the 'Name' column.
Name: pg_stat_statements extension installed
ID: 123456
Category: Extensions
Status: Healthy
Severity: Healthy
Context: The extension pg_stat_statement installed
Impact: Performance - without the extesion it is much harder to detect and analyze DB performance
Remediation Plan: None
